In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [3]:
len_syn = 1308.0216666666665
len_mis = 2616.043333333333
len_lof = 167.616

In [4]:
prop_kept_syn = 32320/247378
prop_kept_mis = 32320/505963
prop_kept_lof = 1

In [ ]:
### AVERAGE SFS

In [5]:
nbins = 13
n=10000

vartypes = ['synonymous', 'missense', 'lof']
centers_geo = ['centerE16N4', 'centerE9N9', 'centerE6N4']
w_list_geo = ['50000', '100000', '150000']
labs_geo = ['A', 'B', 'C', 'D']

centers_pca = ['centerX19Y4']
w_list_pca = [0.0015, 0.0025, 0.005]
labs_pca = ['E', 'F', 'G', 'H']
max_sfs_val=1000
sfs_data = []
for k, vt in enumerate(vartypes):
    print(vt)
    if vt=='synonymous':
        scale_factor = len_syn*prop_kept_syn # also divide by prop kept
    elif vt=='missense':
        scale_factor = len_mis*prop_kept_mis # also divide by prop kept
    elif vt=='lof':
        scale_factor = len_lof
    print(scale_factor)
    
    # Process PCA data
    sfs_unif_pca = pd.read_csv(f'../empirical/subsampling_SIR_v20250127/results/sfs_freq/chr1_{vt}_uniformpca_nSIR10000_nSIRreps10.SIRfreq.sfs', sep='\t')
    sfs_unif_pca_grouped = sfs_unif_pca.groupby('COUNT')['SFS_COUNTS'].agg(['mean', 'std'])
    sfs_unif_pca_avg = sfs_unif_pca_grouped['mean']/scale_factor
    sfs_unif_pca_std = sfs_unif_pca_grouped['std']/scale_factor
    
    for i in range(max_sfs_val):
        sfs_data.append([vt, "H", i, sfs_unif_pca_avg[i], sfs_unif_pca_std[i]])  
    
    for j, w in enumerate(w_list_pca):
        all_sfs_avg = []
        all_sfs_std = []

        for center in centers_pca:
            sfs = pd.read_csv(f'../empirical/subsampling_SIR_v20250127/results/sfs_freq/chr1_{vt}_{center}pca{w}_nSIR10000_nSIRreps10.SIRfreq.sfs', delimiter='\t')
            sfs_avg_grouped = sfs.groupby('COUNT')['SFS_COUNTS'].agg(['mean', 'std'])
            all_sfs_avg.append(sfs_avg_grouped['mean']/scale_factor)
            all_sfs_std.append(sfs_avg_grouped['std']/scale_factor)


        sfs_avg_final = pd.concat(all_sfs_avg, axis=1).values
        sfs_std_final = pd.concat(all_sfs_std, axis=1).values
        
        for i in range(max_sfs_val):
            sfs_data.append([vt, labs_pca[j], i, sfs_avg_final[i].item(), sfs_std_final[i].item()])



        
    # Load and process uniform data for geo
    sfs_unif_geo = pd.read_csv(f'../empirical/subsampling_SIR_v20250127/results/sfs_freq/chr1_{vt}_uniformgeo_nSIR10000_nSIRreps10.SIRfreq.sfs', sep='\t')
    sfs_unif_geo_grouped = sfs_unif_geo.groupby('COUNT')['SFS_COUNTS'].agg(['mean', 'std'])
    sfs_unif_geo_avg = sfs_unif_geo_grouped['mean']/scale_factor
    sfs_unif_geo_std = sfs_unif_geo_grouped['std']/scale_factor

    for i in range(max_sfs_val):
        sfs_data.append([vt, "D", i, sfs_unif_geo_avg[i], sfs_unif_geo_std[i]])

    # Process Gaussian data for geo
    for j, w in enumerate(w_list_geo):
        all_sfs_avg = []
        all_sfs_std = []

        for center in centers_geo:
            sfs = pd.read_csv(f'../empirical/subsampling_SIR_v20250127/results/sfs_freq/chr1_{vt}_{center}geo{w}_nSIR10000_nSIRreps10.SIRfreq.sfs', delimiter='\t')
            sfs_avg_grouped = sfs.groupby('COUNT')['SFS_COUNTS'].agg(['mean', 'std'])
            all_sfs_avg.append(sfs_avg_grouped['mean']/scale_factor)
            all_sfs_std.append(sfs_avg_grouped['std']/scale_factor)
        
        combined_sfs_avg = pd.concat(all_sfs_avg, axis=1)
        combined_sfs_std = pd.concat(all_sfs_std, axis=1)
        sfs_avg_final = combined_sfs_avg.mean(axis=1)
        sfs_std_final = combined_sfs_std.std(axis=1)

        
        for i in range(max_sfs_val):
            sfs_data.append([vt, labs_geo[j], i, sfs_avg_final[i], sfs_std_final[i]])

        

sfs_df = pd.DataFrame(sfs_data, columns=['Variant_Type', 'Panel', 'Count', 'SFS', 'Std'])
sfs_df.to_csv("sfs_binned_summary_unbinned.csv", index=False)

synonymous
170.89337073897704
missense
167.10810974979066
lof
167.616


In [12]:
sfs_df

,Variant_Type,Panel,Count,SFS,Std
0,synonymous,H,0,154.739765,0.192521
1,synonymous,H,1,12.231019,0.222819
2,synonymous,H,2,4.828157,0.149606
3,synonymous,H,3,2.686471,0.140613
4,synonymous,H,4,1.782983,0.103067
...,...,...,...,...,...
23995,lof,C,995,0.000199,0.001089
23996,lof,C,996,0.000199,0.001089
23997,lof,C,997,0.000199,0.001089
23998,lof,C,998,0.0,0.0


In [ ]:
### RATIOS

In [30]:
nbins = 13
n=10000
# sfs_unif = pd.read_csv(f'../empirical/subsampling_SIR_v20250127/results/sfs_freq/chr1_lof_uniformgeo_nSIR10000_nSIRreps10.SIRfreq.sfs', sep='\t')
# sfs_unif_grouped = sfs_unif.groupby('COUNT')['SFS_COUNTS'].agg(['mean', 'std'])
# sfs_unif_avg = sfs_unif_grouped['mean']


vartypes = ['synonymous', 'missense', 'lof']
centers_geo = ['centerE16N4', 'centerE9N9', 'centerE6N4']
w_list_geo = ['50000', '100000', '150000']
labs_geo = ['A', 'B', 'C', 'D']

centers_pca = ['centerX19Y4']
w_list_pca = [0.0015, 0.0025, 0.005]
labs_pca = ['E', 'F', 'G', 'H']


sfs_data = []
for k, vt in enumerate(vartypes):
    print(vt)
    if vt=='synonymous':
        scale_factor = len_syn*prop_kept_syn # also divide by prop kept
    elif vt=='missense':
        scale_factor = len_mis*prop_kept_mis # also divide by prop kept
    elif vt=='lof':
        scale_factor = len_lof
    print(scale_factor)
    
    # Process PCA data
    sfs_unif_pca = pd.read_csv(f'../empirical/subsampling_SIR_v20250127/results/sfs_freq/chr1_{vt}_uniformpca_nSIR10000_nSIRreps10.SIRfreq.sfs', sep='\t')
    for rep in np.arange(1,11):
        sfs_unif_pca_rep = sfs_unif_pca[sfs_unif_pca['ITER']==rep]#.groupby('OBS_N')['COUNT_SITES'].agg(['mean', 'std'])
        sfs_unif_pca_rep['SFS_COUNTS'] = sfs_unif_pca_rep['SFS_COUNTS']/scale_factor
        sfs_unif_pca_rep.reset_index(inplace=True)

        for i in range(max_sfs_val):
            sfs_data.append([vt, "H", rep, 'uniform', i, sfs_unif_pca_rep['SFS_COUNTS'][i]])  
    
        for j, w in enumerate(w_list_pca):
            all_sfs_avg = []
            all_sfs_std = []
    
            for center in centers_pca:
                sfs = pd.read_csv(f'../empirical/subsampling_SIR_v20250127/results/sfs_freq/chr1_{vt}_{center}pca{w}_nSIR10000_nSIRreps10.SIRfreq.sfs', delimiter='\t')
                sfs_rep = sfs[sfs['ITER']==rep]
                sfs_rep['SFS_COUNTS']=sfs_rep['SFS_COUNTS']/scale_factor
                sfs_rep.reset_index(inplace=True)

                for i in range(max_sfs_val):
                    sfs_data.append([vt, labs_pca[j], rep, center, i, sfs_rep['SFS_COUNTS'][i]])  

    # Process geo data
    sfs_unif_geo = pd.read_csv(f'../empirical/subsampling_SIR_v20250127/results/sfs_freq/chr1_{vt}_uniformgeo_nSIR10000_nSIRreps10.SIRfreq.sfs', sep='\t')
    for rep in np.arange(1,11):
        sfs_unif_geo_rep = sfs_unif_geo[sfs_unif_geo['ITER']==rep]#.groupby('OBS_N')['COUNT_SITES'].agg(['mean', 'std'])
        sfs_unif_geo_rep['SFS_COUNTS'] = sfs_unif_geo_rep['SFS_COUNTS']/scale_factor
        sfs_unif_geo_rep.reset_index(inplace=True)

        for i in range(max_sfs_val):
            sfs_data.append([vt, "D", rep, 'uniform', i, sfs_unif_geo_rep['SFS_COUNTS'][i]])  
    
        for j, w in enumerate(w_list_geo):
            all_sfs_avg = []
            all_sfs_std = []
    
            for center in centers_geo:
                sfs = pd.read_csv(f'../empirical/subsampling_SIR_v20250127/results/sfs_freq/chr1_{vt}_{center}geo{w}_nSIR10000_nSIRreps10.SIRfreq.sfs', delimiter='\t')
                sfs_rep = sfs[sfs['ITER']==rep]
                sfs_rep['SFS_COUNTS']=sfs_rep['SFS_COUNTS']/scale_factor
                sfs_rep.reset_index(inplace=True)

                for i in range(max_sfs_val):
                    sfs_data.append([vt, labs_geo[j], rep, center, i, sfs_rep['SFS_COUNTS'][i]])  


synonymous
170.89337073897704
missense
167.10810974979066
lof
167.616


In [31]:
sfs_df_rep = pd.DataFrame(sfs_data,columns=['Variant Type','Panel','Rep','center','count', 'sfs'])

In [32]:
sfs_df_rep

,Variant Type,Panel,Rep,center,count,sfs
0,synonymous,H,1,uniform,0,155.038197
1,synonymous,H,1,uniform,1,12.036745
2,synonymous,H,1,uniform,2,4.880236
3,synonymous,H,1,uniform,3,2.563002
4,synonymous,H,1,uniform,4,1.948583
...,...,...,...,...,...,...
419995,lof,C,10,centerE6N4,995,0.005966
419996,lof,C,10,centerE6N4,996,0.000000
419997,lof,C,10,centerE6N4,997,0.000000
419998,lof,C,10,centerE6N4,998,0.000000


In [33]:
df = sfs_df_rep
df_A = df[df['Panel']=='A']
df_D = df[df['Panel']=='D']
df_ratio = pd.merge(df_A, df_D, on=['Rep', 'count','Variant Type'], suffixes=('_A', '_D'))
df_ratio['ratio'] = df_ratio['sfs_A'] / df_ratio['sfs_D']

In [35]:
df_ratio_grouped = df_ratio.groupby(['Variant Type', 'count']).agg(
    ratio=('ratio', 'mean'),
    stdev=('ratio', 'std')
).reset_index()

In [36]:
df_ratio_grouped.to_csv('ratios_unbinned.csv')

In [37]:
df = sfs_df_rep
df_E = df[df['Panel']=='E']
df_H = df[df['Panel']=='H']
df_ratio = pd.merge(df_E, df_H, on=['Rep', 'count','Variant Type'], suffixes=('_E', '_H'))
df_ratio['ratio'] = df_ratio['sfs_E'] / df_ratio['sfs_H']

In [39]:
df_ratio_grouped = df_ratio.groupby(['Variant Type', 'count']).agg(
    ratio=('ratio', 'mean'),
    stdev=('ratio', 'std')
).reset_index()
df_ratio_grouped.to_csv('ratios_pca_unbinned.csv')

In [40]:
df_ratio_grouped

,Variant Type,count,ratio,stdev
0,lof,0,1.014643,0.001624
1,lof,1,0.865878,0.037421
2,lof,2,0.880121,0.066593
3,lof,3,0.876252,0.088723
4,lof,4,0.858012,0.123283
...,...,...,...,...
2995,synonymous,995,inf,NaN
2996,synonymous,996,0.000000,NaN
2997,synonymous,997,NaN,NaN
2998,synonymous,998,inf,NaN
